In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

In [ ]:
use_log = pd.read_csv("../input/use_log.csv")
customer_master = pd.read_csv("../input/customer_master.csv")
class_master = pd.read_csv("../input/class_master.csv")
campaign_master = pd.read_csv("../input/campaign_master.csv")

In [ ]:
use_log.head(3)

In [ ]:
customer_master.head(3)

In [ ]:
class_master.head(3)

In [ ]:
campaign_master.head(3)

In [ ]:
customer_join = pd.merge(
    customer_master, 
    class_master,
    on="class",
    how="left",
)

customer_join.head(3)

In [ ]:
customer_join = pd.merge(
    customer_join,
    campaign_master,
    on="campaign_id",
    how="left",
)

customer_join.head(3)

In [ ]:
customer_join.isnull().sum()

In [ ]:
pd.pivot_table(
    data=customer_join,
    index="class_name",
    aggfunc="size",
)

In [ ]:
pd.pivot_table(
    data=customer_join,
    index="campaign_name",
    aggfunc="size",
)

In [ ]:
pd.pivot_table(
    data=customer_join,
    index="is_deleted",
    aggfunc="size",
)

In [ ]:
customer_join["end_date"] = pd.to_datetime(customer_join["end_date"])
customer_new = customer_join.loc[
    (customer_join["end_date"] >= pd.to_datetime("20190331")) |
    (customer_join["end_date"].isna())
]

In [ ]:
pd.pivot_table(
    data=customer_new,
    index="class_name",
    aggfunc="size",
)

In [ ]:
pd.pivot_table(
    data=customer_new,
    index="campaign_name",
    aggfunc="size",
)

In [ ]:
pd.pivot_table(
    data=customer_new,
    index="is_deleted",
    aggfunc="size",
)

In [ ]:
use_log["usedate"] = pd.to_datetime(use_log["usedate"])
use_log["usemonth"] = use_log["usedate"].dt.strftime("%Y%m")
use_log.head(3)

In [ ]:
use_log_months = use_log.groupby(["usemonth", "customer_id"], as_index=False).count()
use_log_months.rename(columns={"log_id": "count"}, inplace=True)
del use_log_months["usedate"]
use_log_months.head(3)

In [ ]:
use_log_customer = use_log_months.groupby("customer_id").agg(["mean", "median", "max", "min"])["count"]
use_log_customer = use_log_customer.reset_index(drop=False)
use_log_customer.head(3)

In [ ]:
use_log["weekday"] = use_log["usedate"].dt.weekday
use_log_weekday = use_log.groupby(["customer_id", "usemonth", "weekday"], as_index=False).count()[["customer_id", "usemonth", "weekday", "log_id"]]
use_log_weekday.rename(columns={"log_id": "count"}, inplace=True)
use_log_weekday.head(3)

In [ ]:
use_log_weekday = use_log_weekday.groupby("customer_id", as_index=False).max()[["customer_id", "count"]]

use_log_weekday["routine_flag"] = 0
use_log_weekday["routine_flag"] = use_log_weekday["routine_flag"].where(use_log_weekday["count"] < 4, 1)
use_log_weekday.head(3)

In [ ]:
customer_all = pd.merge( 
    customer_join,
    use_log_customer,
    on="customer_id",
    how="left",
)

customer_all = pd.merge( 
    customer_all,
    use_log_weekday,
    on="customer_id",
    how="left",
)


customer_all.head(3)

In [ ]:
customer_all["end_date"] = customer_all["end_date"].fillna("20190430")
customer_all["end_date"] = pd.to_datetime(customer_all["end_date"])

customer_all["start_date"] = pd.to_datetime(customer_all["start_date"])
customer_all["start_date"] = customer_all["start_date"].dt.strftime("%Y%m%d")
customer_all["start_date"] = pd.to_datetime(customer_all["start_date"])

customer_all.head(3)

In [ ]:
customer_all.dtypes

In [ ]:
customer_all["membership_period"] = customer_all["end_date"] - customer_all["start_date"]
customer_all.head(3)

In [ ]:
import datetime

customer_all["membership_preriod_int"] = customer_all["membership_period"] / datetime.timedelta(days=1)
customer_all.head(3)

In [ ]:
pd.pivot_table(
    data=customer_all,
    index="routine_flag",
    aggfunc="size",
)

In [ ]:
import seaborn as sns

sns.distplot(customer_all["membership_preriod_int"])